# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.  
 
Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [6]:
import pandas as pd
import random
import nltk
from sklearn.model_selection import train_test_split
# make all functions from the other notebook available here 
%run challenge-1.ipynb

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jeanneendres/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jeanneendres/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jeanneendres/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jeanneendres/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# your code here
df = pd.read_csv('sentiment140.csv', header=None)
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
df_subset = df.sample(n=5000, random_state=42)
df_subset

,target,id,date,flag,user,text
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo..."
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem
...,...,...,...,...,...,...
1158200,4,1979246134,Sun May 31 02:13:48 PDT 2009,NO_QUERY,mimikinns,on my way to freedom - feels like i've been tr...
966771,4,1827723007,Sun May 17 11:18:39 PDT 2009,NO_QUERY,Joyhmb3,Minority report I have seen this one but it's ...
47209,0,1677584303,Sat May 02 02:52:50 PDT 2009,NO_QUERY,Trishanson,Oh so tired.. I can't for my pillow and blanke...
1123661,4,1974531134,Sat May 30 13:19:23 PDT 2009,NO_QUERY,DiaZerva,"@satinephoenix not yet, on set but I think it ..."


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [8]:
# method chaining 
df_subset['text_processed'] = df_subset['text'].apply(lambda x: remove_stopwords(stem_and_lemmatize(word_tokenize(clean_up(x)))))
df_subset.head()

,target,id,date,flag,user,text,text_processed
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!,"[chrishasboob, ahhh, hope, ok]"
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo...","[misstoriblack, cool, tweet, app, razr]"
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...,"[tiannachao, know, famili, drama, lame, hey, n..."
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...,"[school, email, open, geographi, stuff, revis,..."
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem,"[upper, airway, problem]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [9]:
from nltk.probability import FreqDist
# retrieve all words from the processes text 
all_words = [word for sublist in df_subset['text_processed'] for word in sublist]
# get frequency 
freq_dist = FreqDist(all_words)
# only top 5000 words
top_words = freq_dist.most_common(5000)
top_words[:15]

[('go', 442),
 ('get', 365),
 ('wa', 353),
 ('day', 340),
 ('thi', 310),
 ('good', 307),
 ('work', 278),
 ('quot', 261),
 ('like', 256),
 ('love', 228),
 ('got', 226),
 ('today', 219),
 ('u', 215),
 ('lol', 207),
 ('time', 203)]

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [10]:
# create functions
def find_features(document, top_words):
    words = set(document)
    features = {}
    for word in top_words:
        features[word] = (word in words)
    return features

#  create set of features 
featuresets = []
for index, row in df_subset.iterrows():
    features = find_features(row['text_processed'], [word[0] for word in top_words])
    # convert to false and true 
    sentiment = (row['target'] == 4)  
    featuresets.append((features, sentiment))

featuresets[:5]

[({'go': False,
   'get': False,
   'wa': False,
   'day': False,
   'thi': False,
   'good': False,
   'work': False,
   'quot': False,
   'like': False,
   'love': False,
   'got': False,
   'today': False,
   'u': False,
   'lol': False,
   'time': False,
   'miss': False,
   'know': False,
   'thank': False,
   'want': False,
   'one': False,
   'realli': False,
   'back': False,
   'still': False,
   'think': False,
   'see': False,
   'watch': False,
   'feel': False,
   'amp': False,
   'well': False,
   'need': False,
   'na': False,
   'im': False,
   'home': False,
   'night': False,
   'hope': True,
   'new': False,
   'come': False,
   'make': False,
   'tri': False,
   'follow': False,
   'look': False,
   'oh': False,
   'ha': False,
   'wish': False,
   'wait': False,
   'sleep': False,
   'morn': False,
   'great': False,
   'last': False,
   'sorri': False,
   'bad': False,
   'happi': False,
   'haha': False,
   'much': False,
   'tomorrow': False,
   'sad': False,
  

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [11]:
# shuffle, dataset is ordered
random.shuffle(featuresets)

# train, test split 
train_set, test_set = train_test_split(featuresets, test_size=0.2, random_state=42)

classifier = nltk.NaiveBayesClassifier.train(train_set)
classifier.show_most_informative_features()

Most Informative Features
                     sad = True            False : True   =     20.3 : 1.0
                    poor = True            False : True   =     13.8 : 1.0
                    cold = True            False : True   =      9.9 : 1.0
                    lost = True            False : True   =      9.9 : 1.0
                  anyway = True             True : False  =      9.6 : 1.0
                     abl = True            False : True   =      9.1 : 1.0
                   super = True            False : True   =      8.4 : 1.0
                     add = True             True : False  =      8.3 : 1.0
                     die = True            False : True   =      8.3 : 1.0
                     cri = True            False : True   =      7.9 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [12]:
nltk.classify.accuracy(classifier, test_set)

0.691

> 0.691

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [13]:
# your code here
# using bigger subset, 10x
df__bigger_subset = df.sample(n=50000, random_state=42)
df__bigger_subset['text_processed'] = df__bigger_subset['text'].apply(lambda x: remove_stopwords(stem_and_lemmatize(word_tokenize(clean_up(x)))))
featuresets = []
for index, row in df__bigger_subset.iterrows():
    features = find_features(row['text_processed'], [word[0] for word in top_words])
    # convert to false and true 
    sentiment = (row['target'] == 4)  
    featuresets.append((features, sentiment))
# shuffle, dataset is ordered
random.shuffle(featuresets)
# train, test split 
train_set, test_set = train_test_split(featuresets, test_size=0.2, random_state=42)

In [14]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [15]:
nltk.classify.accuracy(classifier, test_set)

0.7484

> 0.7484

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here
# literally putting the previous cell in a function

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here